In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.fpm import FPGrowth
import functools

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1654670452059_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
S3_DATA_SOURCE_PATH = "s3://nabil-fikri-9191919122/data-source/2020-Jan.csv"
S3_DATA_SOURCE_PATH_1 = "s3://nabil-fikri-9191919122/data-source/2020-Feb.csv"
S3_DATA_SOURCE_PATH_2 = "s3://nabil-fikri-9191919122/data-source/2019-Dec.csv"
S3_DATA_OUTPUT_PATH = "s3://nabil-fikri-9191919122/data-output/predicted_res/"
S3_DATA_MODEL_OUTPUT_PATH = "s3://nabil-fikri-9191919122/data-output/hasil/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Data Understanding
Size of this Dataset : 1.34 GB
Total row count : 11,954,720 rows
Total column : 9 column
All of the features are categorical, except for Piece feature

In [3]:
# Import the data from CSV
df = spark.read.csv(S3_DATA_SOURCE_PATH, header=True)
df2 = spark.read.csv(S3_DATA_SOURCE_PATH_1, header=True)
df3 = spark.read.csv(S3_DATA_SOURCE_PATH_2, header=True)
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+----------+-------------------+-------------+---------+------+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code|    brand| price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+---------+------+---------+--------------------+
|2020-01-01 00:00:...|            view|   5809910|1602943681873052386|         null|  grattol|  5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 00:00:...|            view|   5812943|1487580012121948301|         null| kinetics|  3.97|595414640|c8c5205d-be43-4f1...|
|2020-01-01 00:00:...|            view|   5798924|1783999068867920626|         null|   zinger|  3.97|595412617|46a5010f-bd69-4fb...|
|2020-01-01 00:00:...|            view|   5793052|1487580005754995573|         null|     null|  4.92|420652863|546f6af3-a517-475...|
|2020-01-01 00:00:...|            view|   5899926|2115334439910245200

In [4]:
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+----------+-------------------+-------------+------+-----+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code| brand|price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+------+-----+---------+--------------------+
|2020-02-01 00:00:...|            cart|   5844305|1487580006317032337|         null|  null| 2.14|485174092|4be9643a-420b-4c6...|
|2020-02-01 00:00:...|            view|   5769925|1487580013841613016|         null|kapous| 4.22|594621622|a88baf11-9cd0-436...|
|2020-02-01 00:00:...|            view|   5817765|1487580008246412266|         null|zeitun|11.03|495404942|3a569c8d-d848-4f0...|
|2020-02-01 00:00:...|            view|   5877033|1487580010100293687|         null|  milv| 3.49|564814969|7feb39e5-bb7b-4b2...|
|2020-02-01 00:00:...|            cart|   5814871|1487580008112194531|         null|zinger| 2.54|

In [5]:
df3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+----------+-------------------+-------------+---------+-----+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code|    brand|price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+---------+-----+---------+--------------------+
|2019-12-01 00:00:...|remove_from_cart|   5712790|1487580005268456287|         null|    f.o.x| 6.27|576802932|51d85cb0-897f-48d...|
|2019-12-01 00:00:...|            view|   5764655|1487580005411062629|         null|      cnd|29.05|412120092|8adff31e-2051-489...|
|2019-12-01 00:00:...|            cart|      4958|1487580009471148064|         null|   runail| 1.19|494077766|c99a50e8-2fac-4c4...|
|2019-12-01 00:00:...|            view|   5848413|1487580007675986893|         null|freedecor| 0.79|348405118|722ffea5-73c0-492...|
|2019-12-01 00:00:...|            view|   5824148|1487580005511725929|      

# Data Preparation
The source data is saved in s3://nabil-fikri-9191919122/data-source/

## Data Preprocessing and Feature Engineering

In [6]:
# explicit function
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
unioned_df = unionAll([df, df2, df3])
unioned_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+----------+-------------------+-------------+---------+------+---------+--------------------+
|          event_time|      event_type|product_id|        category_id|category_code|    brand| price|  user_id|        user_session|
+--------------------+----------------+----------+-------------------+-------------+---------+------+---------+--------------------+
|2020-01-01 00:00:...|            view|   5809910|1602943681873052386|         null|  grattol|  5.24|595414620|4adb70bb-edbd-498...|
|2020-01-01 00:00:...|            view|   5812943|1487580012121948301|         null| kinetics|  3.97|595414640|c8c5205d-be43-4f1...|
|2020-01-01 00:00:...|            view|   5798924|1783999068867920626|         null|   zinger|  3.97|595412617|46a5010f-bd69-4fb...|
|2020-01-01 00:00:...|            view|   5793052|1487580005754995573|         null|     null|  4.92|420652863|546f6af3-a517-475...|
|2020-01-01 00:00:...|            view|   5899926|2115334439910245200

In [8]:
df = unioned_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#select purchase row
df_raw = df.where((col('event_type') == 'purchase'))
print("Source total row: %s" % df.count())
print("Purchase only total row: %s" % df_raw.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Source total row: 11954720
Purchase only total row: 718966

In [10]:
# Select 3 column
df_raw = df_raw.select("event_time", "product_id", "user_id")
df_raw.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+---------+
|          event_time|product_id|  user_id|
+--------------------+----------+---------+
|2020-01-01 00:06:...|   5802440|595411904|
|2020-01-01 00:06:...|   5825203|595411904|
|2020-01-01 00:51:...|   5842655|595351517|
|2020-01-01 00:52:...|      4185|287758725|
|2020-01-01 00:52:...|   5700088|504269664|
|2020-01-01 00:52:...|   5838656|504269664|
|2020-01-01 00:52:...|   5723410|504269664|
|2020-01-01 00:52:...|   5700095|504269664|
|2020-01-01 00:52:...|   5700094|504269664|
|2020-01-01 00:52:...|   5700090|504269664|
|2020-01-01 00:58:...|     59003|336065468|
|2020-01-01 00:58:...|     59002|336065468|
|2020-01-01 00:58:...|   5869135|336065468|
|2020-01-01 00:58:...|   5827374|336065468|
|2020-01-01 00:58:...|   5827006|336065468|
|2020-01-01 00:58:...|   5813830|336065468|
|2020-01-01 00:58:...|   5802546|336065468|
|2020-01-01 00:58:...|   5798928|336065468|
|2020-01-01 00:58:...|   5798927|336065468|
|2020-01-01 00:58:...|   5774342

In [11]:
df_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

718966

In [12]:
# check is there is empty, None, Null, Nan with string literals.
df2 = df_raw.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_raw.select('event_time', 'product_id', 'user_id').columns])
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+-------+
|event_time|product_id|user_id|
+----------+----------+-------+
|         0|         0|      0|
+----------+----------+-------+

In [13]:
#check if there is user_id have different event_time in purchasing
df.select("event_time", "user_id").groupBy('user_id').count().where((col('count') > 1)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----+
|  user_id|count|
+---------+-----+
|413759078|   13|
|595605641|   19|
|572606397|    9|
|588856025|   43|
|595693462|    2|
|454356614|    7|
|539907585|  229|
|594871354|   26|
|581581563|  828|
|475829132|   22|
|593589937|   23|
|595852644|    2|
|504629532|   23|
|595994594|    3|
|596083934|    3|
|594036817|    2|
|572401854|   21|
|596216560|    5|
|596241412|    3|
|566406961|  162|
+---------+-----+
only showing top 20 rows

In [14]:
# if same
# df_raw = df_raw.select("user_id", "product_id")
# df_raw = df_raw.withColumnRenamed("user_id", 'id')

# if not same (convert event_time to seconds and join seconds with user_id)
df_raw = df_raw.withColumn('event_time', regexp_replace('event_time', ' UTC', ''))
df_raw = df_raw.withColumn('event_time', to_timestamp(df_raw.event_time, 'yyyy-MM-dd HH:mm:ss'))
df_raw = df_raw.withColumn('end_timestamp', current_timestamp())\
            .withColumn('seconds', col('end_timestamp').cast('long')-col('event_time').cast('long'))
df_raw = df_raw.withColumn('id', (df_raw.seconds*1000000000)+df_raw.user_id.cast('long'))
df_raw = df_raw.select('id', 'product_id')

df_raw.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+
|               id|product_id|
+-----------------+----------+
|76840273595411904|   5802440|
|76840273595411904|   5825203|
|76837542595351517|   5842655|
|76837520287758725|      4185|
|76837509504269664|   5700088|
|76837509504269664|   5838656|
|76837509504269664|   5723410|
|76837509504269664|   5700095|
|76837509504269664|   5700094|
|76837509504269664|   5700090|
|76837130336065468|     59003|
|76837130336065468|     59002|
|76837130336065468|   5869135|
|76837130336065468|   5827374|
|76837130336065468|   5827006|
|76837130336065468|   5813830|
|76837130336065468|   5802546|
|76837130336065468|   5798928|
|76837130336065468|   5798927|
|76837130336065468|   5774342|
+-----------------+----------+
only showing top 20 rows

In [15]:
# Before dropping the duplicates
df_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

718966

In [16]:
# Remove the duplicates
df_raw = df_raw.dropDuplicates(["id", "product_id"])
df_raw.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

718513

In [17]:
# Convert into this format: ["id", "items"] = [1, [a, b, c]]

df_input = df_raw.groupBy("id").agg(collect_list('product_id').alias('items'))
df_input.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+
|               id|               items|
+-----------------+--------------------+
|71667288549936053|[5739988, 5813878...|
|71667319622045495|[5712544, 5661520...|
|71669977622042844|[5924419, 6846, 5...|
|71671111609948609|[5881438, 5881450...|
|71673171611613605|[5822340, 5822355...|
|71674565604511140|[5685798, 5620922...|
|71674699571706189|[5855117, 5861294...|
|71674945467917233|[5817691, 5859210...|
|71676228610072586|  [5896330, 5650627]|
|71678324515512813|[5697021, 5895884...|
|71678517615170409|[5763238, 6495, 5...|
|71682003244605896|[5759385, 5863095...|
|71683921619309094|[5875318, 5875320...|
|71685367619873968|[5759492, 5743974...|
|71688326615985575|[5797793, 5797794...|
|71690123603212651|           [5907080]|
|71694936532611936|[5766979, 5829354...|
|71696052465095444|           [5854897]|
|71696226597851470|           [5885594]|
|71698451621868090|[5898268, 5898262...|
+-----------------+--------------------+
only showing top

In [18]:
df_input.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

92243

# Modeling

I choose this algorithm because the needed minSupport is low so it can run fast, enough to represent the objective goal, and this algorithm is what i most familiar with.

In [19]:
# Train the data
# 1. Create the instance
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.01) #hyper parameter
# 2. Train/fit the data training to become a model
model = fpGrowth.fit(df_input)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Evaluation

At start i use minSupport at 0.1 and minConfidence at 0.1, but the output is very low.
After i decrease the minSupport to 0.001 the output become higher.
I think because in this cosmetic transaction is still highly dominated by several product, fairly uneven.

In [20]:
model.freqItemsets.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----+
|             items|freq|
+------------------+----+
|         [5870111]| 297|
|         [5834736]| 213|
|         [5862688]| 163|
|         [5569528]| 136|
|         [5820744]| 116|
|         [5682471]| 103|
|         [5809910]|5100|
|         [5888070]| 136|
|         [5889183]| 116|
|         [5861113]| 103|
|         [5854897]|2765|
|[5854897, 5809910]| 229|
|         [5736284]| 295|
|           [60183]| 213|
|         [5802162]| 163|
|         [5809912]|2117|
|[5809912, 5854897]| 100|
|[5809912, 5809910]|1045|
|         [5745313]| 295|
|         [5802650]| 212|
+------------------+----+
only showing top 20 rows

/mnt/yarn/usercache/livy/appcache/application_1654670452059_0004/container_1654670452059_0004_01_000001/pyspark.zip/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.

In [21]:
model.associationRules.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+-------------------+------------------+--------------------+
|          antecedent|consequent|         confidence|              lift|             support|
+--------------------+----------+-------------------+------------------+--------------------+
|              [5013]| [5854897]|0.15717092337917485| 5.243369795755958|0.001734548963064948|
|              [5013]| [5700037]|0.09233791748526522| 4.029104315323235|0.001019047515800...|
|              [5013]| [5809910]| 0.0962671905697446| 1.741171462691167|0.001062411239877...|
|              [5013]|    [4938]| 0.1031434184675835|13.650298923537022|0.001138297757011...|
|              [5013]| [5857360]|0.12868369351669942|18.489361278911066|0.001420161963509...|
|  [5584840, 5584844]| [5584838]| 0.8467741935483871|333.79911083540117|0.001138297757011...|
|  [5584840, 5584844]| [5584836]| 0.8709677419354839|266.91254956596293| 0.00117082055006884|
|           [5857007]| [5817702]| 0.1769352290679305|22.1452

# Deployment

The predicted result is saved in s3://nabil-fikri-9191919122/data-output/predicted_res/
The model is saved in s3://nabil-fikri-9191919122/data-output/hasil/

In [24]:
# Prepare the input test
test = spark.createDataFrame(
    [
     ("123", ["5809910", "5854897"]),
     ("456", ["55854897", "5823667", "5854899"]),
     ("666", ["5854899"]),
     ("124", ["5809910"]),
     ("125", ["5854897"]),
    ],
    ["id", "items"]
)

test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+
| id|               items|
+---+--------------------+
|123|  [5809910, 5854897]|
|456|[55854897, 582366...|
|666|           [5854899]|
|124|           [5809910]|
|125|           [5854897]|
+---+--------------------+

In [25]:
# Predict the input based on the model
result = model.transform(test)
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+--------------------+
| id|               items|          prediction|
+---+--------------------+--------------------+
|123|  [5809910, 5854897]|[5809912, 5700037...|
|456|[55854897, 582366...|  [5819894, 5836517]|
|666|           [5854899]|           [5836517]|
|124|           [5809910]|[5854897, 5809912...|
|125|           [5854897]|[5809910, 5809912...|
+---+--------------------+--------------------+

In [26]:
# Convert array of values into rows
result = result.select(result.id, result.items, explode(result.prediction).alias("prediction"))
result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+----------+
| id|             items|prediction|
+---+------------------+----------+
|123|[5809910, 5854897]|   5809912|
|123|[5809910, 5854897]|   5700037|
|123|[5809910, 5854897]|   5802432|
|123|[5809910, 5854897]|   5751422|
|123|[5809910, 5854897]|   5815662|
|123|[5809910, 5854897]|      5304|
|123|[5809910, 5854897]|   5849033|
|123|[5809910, 5854897]|   5751383|
|123|[5809910, 5854897]|   5792800|
|123|[5809910, 5854897]|   5686925|
|123|[5809910, 5854897]|   5816170|
|123|[5809910, 5854897]|   5809911|
|123|[5809910, 5854897]|      5013|
|123|[5809910, 5854897]|   5687151|
|123|[5809910, 5854897]|   5862943|
|123|[5809910, 5854897]|   5789668|
|123|[5809910, 5854897]|   5814046|
|123|[5809910, 5854897]|   5649219|
|123|[5809910, 5854897]|   5816166|
|123|[5809910, 5854897]|   5816172|
+---+------------------+----------+
only showing top 20 rows

In [27]:
# Save original dataframe into temporary table
df.createOrReplaceTempView("transactions")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# You can use Spark SQL Context to do query
# In this query, we want to select the StockCode and Description and save it into products table
spark.sql("SELECT DISTINCT product_id, brand, price FROM transactions").createOrReplaceTempView("products")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
spark.sql("SELECT * FROM products").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+-----+
|product_id|      brand|price|
+----------+-----------+-----+
|   5622678|   severina| 9.11|
|   5646937|       null| 5.56|
|   5895781|       null| 1.21|
|   5857005|     runail| 3.97|
|   5701880|    bluesky| 2.97|
|   5645616|        cnd|16.03|
|   5894905|   jessnail| 1.10|
|   5813856|       null| 3.97|
|   5778113|     zinger| 5.40|
|   5813485|     masura| 1.73|
|   5815777|beauty-free| 2.37|
|   5875354|       null| 2.05|
|   5697334|       null| 1.84|
|   5861347|       null| 3.65|
|   5803102|       null| 3.97|
|   5911777|  dartnails| 1.27|
|   5850570|       null| 6.35|
|   5848082|       null| 9.52|
|   5909983|      dewal|48.19|
|   5797137|       null| 4.44|
+----------+-----------+-----+
only showing top 20 rows

In [30]:
# Create temporary table from result DataFrame
result.createOrReplaceTempView("result")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
to_output = spark.sql("SELECT r.id, r.items, r.prediction, p.brand, p.price FROM result r INNER JOIN products p ON r.prediction = p.product_id ORDER BY r.id")
to_output.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------------------+----------+--------+-----+
| id|             items|prediction|   brand|price|
+---+------------------+----------+--------+-----+
|123|[5809910, 5854897]|   5816166| grattol| 5.24|
|123|[5809910, 5854897]|   5816166|    null| 0.00|
|123|[5809910, 5854897]|   5687151|    null| 1.90|
|123|[5809910, 5854897]|   5687151|    null| 0.00|
|123|[5809910, 5854897]|   5816173|    null| 0.00|
|123|[5809910, 5854897]|   5816173| grattol| 5.24|
|123|[5809910, 5854897]|   5793703|    null| 0.00|
|123|[5809910, 5854897]|   5793703|   irisk| 2.22|
|123|[5809910, 5854897]|   5751383|     uno| 8.76|
|123|[5809910, 5854897]|   5751383|    null| 0.00|
|123|[5809910, 5854897]|   5751383|     uno|10.32|
|123|[5809910, 5854897]|   5751383|     uno| 8.78|
|123|[5809910, 5854897]|   5809911| grattol| 5.24|
|123|[5809910, 5854897]|   5809911|    null| 0.00|
|123|[5809910, 5854897]|   5843836|    null| 0.00|
|123|[5809910, 5854897]|   5843836|    null| 0.38|
|123|[5809910, 5854897]|   5792

In [32]:
to_output.printSchema()
to_output.withColumn('items', col('items').cast('string')).write.csv(S3_DATA_OUTPUT_PATH)
# print(type(to_output))
print('Predicted Result was successfully saved to s3: %s' % S3_DATA_OUTPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- prediction: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: string (nullable = true)

Predicted Result was successfully saved to s3: s3://nabil-fikri-9191919122/data-output/predicted_res/

In [33]:
model.save(S3_DATA_MODEL_OUTPUT_PATH)
print('Model was successfully saved.')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model was successfully saved.